In [1]:
import os
import SimpleITK as sitk
import numpy as np
from annotatepropwizard3d.mask_propagation import MaskPropagation
from skimage.measure import regionprops
import torch
import gdown

/volume/willy-dev/sota/tmp/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = MaskPropagation('../weights/sam_vit_b_01ec64.pth', 
                        '../yaml/eval_config.yaml')

/volume/willy-dev/sota/tmp/.venv/lib/python3.11/site-packages/annotatepropwizard3d/segment_anything/build_sam.py:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dic

*******load ../weights/sam_vit_b_01ec64.pth


INFO:root:Single object: False
INFO:root:Object transformer enabled: True
/volume/willy-dev/sota/tmp/.venv/lib/python3.11/site-packages/annotatepropwizard3d/mask_propagation.py:91: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub f

In [3]:
os.makedirs('data', exist_ok=True)

output = 'data/amos_0001_data.nii.gz'
if not os.path.exists(output):
    file_id = '1TtQyLI0X6nq90n0dw3zJFFZZzgrZj8br'
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, output)

output = 'data/amos_0001_label.nii.gz'
if not os.path.exists(output):
    file_id = '1MgfbufE3802ZsNwQO8Xloce1ezXMszpM'
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, output)

In [ ]:
sitk_img = sitk.ReadImage('data/amos_0001_data.nii.gz')
img = sitk.GetArrayFromImage(sitk_img)

sitk_mask = sitk.ReadImage('data/amos_0001_label.nii.gz')
mask = sitk.GetArrayFromImage(sitk_mask)

mask = mask.astype(int)
mask[mask != 1] = 0
mask[mask == 1] = 1

mask = mask.astype(np.uint8)
labeled_mask = np.zeros_like(mask)

for prop in regionprops(mask):

    init_center = np.array(prop.centroid, dtype=int)
    index_z = init_center[0]
    labeled_mask[index_z] = mask[index_z]

with torch.inference_mode():
    result = model.predict_by_volume(img, labeled_mask)

f1 = 2 * (result * mask).sum() / (result.sum() + mask.sum())
print(f"f1 score is: {f1}")